In [ ]:
#Smaller Batch Size + More Epochs

In [ ]:
import torch
import numpy as np
import pandas as pd
import time
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


dataset = load_dataset("csv", data_files={"train": "ISEAR_train.csv", "test": "ISEAR_test.csv"})


unique_emotions = sorted(set(dataset["train"]["emotion"]))
emotion2label = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
num_labels = len(emotion2label)
print("Emotion to label mapping:", emotion2label)

def map_emotion_to_label(example):
    example["label"] = emotion2label[example["emotion"]]
    return example

dataset = dataset.map(map_emotion_to_label)


roberta_model_name = "roberta-base"
tokenizer_roberta = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name, num_labels=num_labels)


def tokenize_roberta(examples):
    return tokenizer_roberta(examples["text"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(tokenize_roberta, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

roberta_training_args = TrainingArguments(
    output_dir="./roberta_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,  # Keep best two checkpoints
    load_best_model_at_end=True,  # Load the best checkpoint at the end
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    metric_for_best_model="f1",  # Monitor validation loss
    greater_is_better=True,  # Minimize loss
    save_on_each_node=False
)


roberta_trainer = Trainer(
    model=roberta_model,
    args=roberta_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,

)


start_time = time.time()
roberta_trainer.train()

end_time = time.time()
training_time = end_time - start_time
print(f" Training time: {training_time / 60:.2f} minutes")


roberta_results = roberta_trainer.evaluate()
print(" RoBERTa evaluation results:")
print(roberta_results)


Emotion to label mapping: {'anger': 0, 'disgust': 1, 'fear': 2, 'guilt': 3, 'joy': 4, 'sadness': 5, 'shame': 6}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/jovyan/.local/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training RoBERTa model...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.977900,0.819793,0.720627,0.731045,0.720627,0.722968
2,0.742900,0.787445,0.740209,0.741698,0.740209,0.739265
3,0.552800,0.829255,0.730418,0.734893,0.730418,0.731008
4,0.356200,0.943709,0.734987,0.735715,0.734987,0.733948
5,0.296800,1.032783,0.728460,0.731519,0.728460,0.728681
6,0.210700,1.237035,0.729765,0.737410,0.729765,0.731677
7,0.127800,1.525490,0.729765,0.736934,0.729765,0.731381
8,0.125100,1.626105,0.731723,0.741604,0.731723,0.733787
9,0.097100,1.696814,0.735640,0.739048,0.735640,0.736520
10,0.073300,1.787460,0.729112,0.732548,0.729112,0.728454


🕒 Training time: 11.27 minutes
Evaluating RoBERTa model...


📊 RoBERTa evaluation results:
{'eval_loss': 1.7967922687530518, 'eval_accuracy': 0.7558746736292428, 'eval_precision': 0.7562959357835896, 'eval_recall': 0.7558746736292428, 'eval_f1': 0.7558340673261676, 'eval_runtime': 2.6208, 'eval_samples_per_second': 584.564, 'eval_steps_per_second': 36.631, 'epoch': 15.0}
